In [1]:
!pip install streamlit pyngrok tensorflow pillow matplotlib seaborn scikit-learn opencv-python-headless


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 152.6 MB/s eta 0:00:00


In [2]:
!pip install fpdf


  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=ec47a21d1fe293704728a37b53a1cf4ccb545b9f54912d99eaac864cf706dd4d
  Stored in directory: /root/.cache/pip/wheels/6e/62/11/dc73d78e40a218ad52e7451f30166e94491be013a7850b5d75
Successfully built fpdf


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install reportlab


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 52.4 MB/s eta 0:00:00


In [5]:
!pip install streamlit pyngrok


In [6]:
!pip install psutil

In [8]:
%%writefile app5.py
import streamlit as st
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import cv2
import os
import pandas as pd
from datetime import datetime
import io
import json

# -----------------------------
# Page Setup
# -----------------------------
st.set_page_config(page_title="Brain Tumour Detection & Segmentation", layout="wide")
st.title("🧠 Brain Tumour Classification & Segmentation App")

# -----------------------------
# Load Pre-trained Models Automatically
# -----------------------------
st.sidebar.header("⚙️ Model Loading Status")

# Update these paths as needed
clf_model_path = "/content/drive/MyDrive/models/vgg16_brain_tumor_classification_model_freeze_layer_7.keras"
seg_model_path = "/content/drive/MyDrive/models/segmentation_unet.h5"

clf_model, seg_model = None, None

# --- Load Classification Model ---
if os.path.exists(clf_model_path):
    try:
        clf_model = load_model(clf_model_path)
        st.sidebar.success("✅ Classification model loaded successfully!")
    except Exception as e:
        st.sidebar.error(f"❌ Failed to load classification model: {e}")
else:
    st.sidebar.error(f"❌ Classification model not found at: {clf_model_path}")

# --- Load Segmentation Model ---
if os.path.exists(seg_model_path):
    try:
        seg_model = load_model(seg_model_path)
        st.sidebar.success("✅ Segmentation model loaded successfully!")
    except Exception as e:
        st.sidebar.error(f"❌ Failed to load segmentation model: {e}")
else:
    st.sidebar.warning(f"⚠️ Segmentation model not found at: {seg_model_path}")

# -----------------------------
# Utility Functions
# -----------------------------
def calculate_confidence_score(prediction):
    """Calculate confidence score from prediction probabilities"""
    return float(np.max(prediction)) * 100

def calculate_bounding_box(mask):
    """Calculate bounding box coordinates from segmentation mask"""
    if mask.ndim == 3:
        mask = mask.squeeze()

    # Find coordinates where mask is 1
    coords = np.column_stack(np.where(mask > 0))

    if len(coords) == 0:
        return None

    # Get bounding box coordinates
    y_min, x_min = coords.min(axis=0)
    y_max, x_max = coords.max(axis=0)

    return {
        'x_min': int(x_min),
        'y_min': int(y_min),
        'x_max': int(x_max),
        'y_max': int(y_max),
        'width': int(x_max - x_min),
        'height': int(y_max - y_min)
    }

def draw_bounding_box(img_array, bbox, color=(0, 255, 0), thickness=2):
    """Draw bounding box on image"""
    img_copy = (img_array * 255).astype(np.uint8) if img_array.max() <= 1 else img_array.copy()

    if bbox:
        cv2.rectangle(img_copy,
                     (bbox['x_min'], bbox['y_min']),
                     (bbox['x_max'], bbox['y_max']),
                     color, thickness)

    return img_copy

def generate_automatic_clinical_notes(tumour_type, confidence_score, tumour_area, bounding_box):
    """Generate automatic clinical notes based on scan results"""

    if tumour_type == "No Tumour":
        return f"Automated analysis indicates NO TUMOUR DETECTED. The MRI scan appears normal with no signs of pathological mass. Confidence level: {confidence_score:.1f}%."

    # For tumour cases
    notes = f"Automated analysis detected {tumour_type.upper()} with {confidence_score:.1f}% confidence. "

    # Add size information
    if tumour_area > 0:
        approx_cm2 = tumour_area * 0.0025
        if tumour_area < 1000:
            size_desc = "small"
        elif tumour_area < 5000:
            size_desc = "medium-sized"
        else:
            size_desc = "large"
        notes += f"The tumour appears {size_desc} with an estimated area of {tumour_area} pixels ({approx_cm2:.2f} cm²). "

    # Add location information from bounding box
    if bounding_box:
        center_x = (bounding_box['x_min'] + bounding_box['x_max']) // 2
        center_y = (bounding_box['y_min'] + bounding_box['y_max']) // 2

        # Determine approximate location
        if center_x < 75:
            horiz_loc = "left hemisphere"
        elif center_x > 150:
            horiz_loc = "right hemisphere"
        else:
            horiz_loc = "central region"

        if center_y < 75:
            vert_loc = "superior"
        elif center_y > 150:
            vert_loc = "inferior"
        else:
            vert_loc = "mid"

        notes += f"Located in the {vert_loc} {horiz_loc}. "

    # Add tumour-specific notes
    if tumour_type == "Glioma":
        notes += "Gliomas typically arise from glial cells and may require further evaluation for grading."
    elif tumour_type == "Meningioma":
        notes += "Meningiomas are usually extra-axial tumours arising from meningeal layers."
    elif tumour_type == "Pituitary":
        notes += "Pituitary tumours may affect hormonal functions and require endocrine evaluation."

    notes += " Recommend correlation with clinical findings and consideration of follow-up imaging."
    return notes

def generate_text_report(report_data):
    """Generate a text format report"""

    # Generate automatic notes if not provided by user
    clinical_notes = report_data['clinical_notes']
    if clinical_notes == "No significant clinical history noted.":
        clinical_notes = generate_automatic_clinical_notes(
            report_data['tumour_type'],
            report_data['confidence_score'],
            report_data['tumour_area'],
            report_data['bounding_box']
        )

    report = f"""
BRAIN TUMOUR ANALYSIS REPORT
============================

Patient Information:
-------------------
Patient ID: {report_data['patient_id']}
Age: {report_data['age']}
Gender: {report_data['gender']}
Scan Date: {report_data['scan_date']}

SCAN ANALYSIS RESULTS:
---------------------
Tumour Type: {report_data['tumour_type']}
Confidence Score: {report_data['confidence_score']:.2f}%
Tumour Area: {report_data['tumour_area']} pixels
Approximate Area: {(report_data['tumour_area'] * 0.0025):.2f} cm²

SCAN INTERPRETATION:
-------------------
{'✅ NO TUMOUR DETECTED - Scan appears normal' if report_data['tumour_type'] == 'No Tumour' else f'🚨 ABNORMAL SCAN - {report_data["tumour_type"]} detected'}

Bounding Box Information:
------------------------
{json.dumps(report_data['bounding_box'], indent=2) if report_data['bounding_box'] else 'No tumour detected - No bounding box available'}

AUTOMATED CLINICAL NOTES:
------------------------
{clinical_notes}

TECHNICAL DETAILS:
------------------
Analysis performed using deep learning models for brain tumour classification and segmentation.
Report Generated on: {report_data['report_date']}

DISCLAIMER:
-----------
This is an automated analysis report. Results should be verified by a qualified radiologist.
The findings are based on AI analysis and should be correlated with clinical symptoms and other diagnostic tests.
"""
    return report

# -----------------------------
# Image Upload
# -----------------------------
uploaded_image = st.file_uploader("🩻 Upload MRI Image for Prediction", type=["jpg", "jpeg", "png"])

# Initialize session state for report data
if 'report_data' not in st.session_state:
    st.session_state.report_data = None

if uploaded_image is not None and clf_model is not None:
    # Read and preprocess image
    img = image.load_img(uploaded_image, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0
    img_expanded = np.expand_dims(img_array, axis=0)

    # -----------------------------
    # Classification
    # -----------------------------
    st.subheader("🧩 Step 1: Tumour Classification")

    prediction = clf_model.predict(img_expanded, verbose=0)
    pred_class = np.argmax(prediction, axis=1)[0]

    labels = ["Glioma", "Meningioma", "No Tumour", "Pituitary"]
    tumour_type = labels[pred_class]
    confidence_score = calculate_confidence_score(prediction)

    # Display results with appropriate styling
    if tumour_type == "No Tumour":
        st.success(f"### 🧠 Scan Result: **NO TUMOUR DETECTED**")
    else:
        st.warning(f"### 🚨 Scan Result: **{tumour_type} DETECTED**")

    st.markdown(f"### 📊 Confidence Score: **{confidence_score:.2f}%**")

    # Display confidence scores for all classes
    st.subheader("📈 Detailed Confidence Scores")
    conf_df = pd.DataFrame({
        'Tumour Type': labels,
        'Confidence Score': [f"{score:.2f}%" for score in prediction[0] * 100]
    })
    st.dataframe(conf_df)

    # -----------------------------
    # Segmentation (if tumour found)
    # -----------------------------
    bounding_box = None
    tumour_area = 0
    mask_resized = None
    bbox_image = None

    if tumour_type != "No Tumour" and seg_model is not None:
        st.subheader("🔍 Step 2: Tumour Segmentation")

        # --- Get model input shape dynamically ---
        model_input_shape = seg_model.input_shape  # (None, H, W, C)
        _, h, w, c = model_input_shape

        # --- Preprocess image for segmentation ---
        img_np = np.array(img)

        # If model expects grayscale
        if c == 1:
            img_gray = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)
            seg_input = cv2.resize(img_gray, (w, h)) / 255.0
            seg_input = np.expand_dims(seg_input, axis=-1)
        else:
            seg_input = cv2.resize(img_np, (w, h)) / 255.0

        seg_input = np.expand_dims(seg_input, axis=0)

        # --- Predict mask ---
        mask = seg_model.predict(seg_input, verbose=0)[0]
        mask = (mask > 0.5).astype(np.uint8)

        # Resize mask for overlay visualization
        mask_resized = cv2.resize(mask, (224, 224))

        # Calculate tumour area
        tumour_area = np.sum(mask_resized)

        # Calculate bounding box
        bounding_box = calculate_bounding_box(mask_resized)

        # Create image with bounding box
        if bounding_box:
            bbox_image = draw_bounding_box((img_array * 255).astype(np.uint8), bounding_box)

        # Overlay mask on image
        overlay = img_array.copy()
        if mask_resized.ndim == 3:
            mask_resized_vis = mask_resized.squeeze()
        else:
            mask_resized_vis = mask_resized
        overlay[mask_resized_vis == 1] = [1, 0, 0]  # red overlay

        # --- Visualization ---
        st.subheader("🖼️ Visualization Results")
        col1, col2, col3, col4 = st.columns(4)
        with col1:
            st.image(img_array, caption="Original MRI Scan", use_container_width=True)
        with col2:
            st.image(mask_resized * 255, caption="Tumour Segmentation", use_container_width=True)
        with col3:
            st.image(overlay, caption="Tumour Overlay", use_container_width=True)
        with col4:
            if bbox_image is not None:
                st.image(bbox_image, caption="Tumour Localization", use_container_width=True)
            else:
                st.image(img_array, caption="No Tumour Detected", use_container_width=True)

        # --- Tumour area and bounding box info ---
        st.subheader("📏 Quantitative Analysis")

        col1, col2 = st.columns(2)
        with col1:
            st.metric("Estimated Tumour Area (pixels)", f"{tumour_area}")
            st.metric("Tumour Area (cm² approx)", f"{(tumour_area * 0.0025):.2f}")

        with col2:
            if bounding_box:
                st.metric("Bounding Box Width", f"{bounding_box['width']} px")
                st.metric("Bounding Box Height", f"{bounding_box['height']} px")

        if bounding_box:
            st.subheader("📐 Tumour Location Details")
            st.json(bounding_box)

    elif tumour_type == "No Tumour":
        st.success("✅ No tumour detected — segmentation skipped.")
        tumour_area = 0
        bounding_box = None
    elif seg_model is None:
        st.warning("⚠️ Please make sure the segmentation model file is present.")

    # -----------------------------
    # Report Generation Section
    # -----------------------------
    st.subheader("📄 Generate Analysis Report")

    with st.form("patient_info_form"):
        st.write("### Patient Information")
        col1, col2 = st.columns(2)

        with col1:
            patient_id = st.text_input("Patient ID", "PAT_001")
            age = st.number_input("Age", min_value=0, max_value=120, value=45)

        with col2:
            gender = st.selectbox("Gender", ["Male", "Female", "Other"])
            scan_date = st.date_input("Scan Date", datetime.now())

        clinical_notes = st.text_area("Clinical Notes (Optional)", "No significant clinical history noted.")

        generate_report = st.form_submit_button("📊 Generate Comprehensive Report")

    if generate_report:
        # Generate automatic clinical notes
        auto_notes = generate_automatic_clinical_notes(tumour_type, confidence_score, tumour_area, bounding_box)

        # Use automatic notes if user didn't provide specific ones
        final_notes = auto_notes if clinical_notes == "No significant clinical history noted." else clinical_notes

        report_data = {
            'report_date': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            'patient_id': patient_id,
            'age': age,
            'gender': gender,
            'scan_date': str(scan_date),
            'clinical_notes': final_notes,
            'tumour_type': tumour_type,
            'confidence_score': confidence_score,
            'tumour_area': int(tumour_area),
            'bounding_box': bounding_box,
            'scan_result': 'NORMAL' if tumour_type == 'No Tumour' else 'ABNORMAL',
            'analysis_summary': f"Detected {tumour_type} with {confidence_score:.2f}% confidence. Tumour area: {tumour_area} pixels."
        }

        st.session_state.report_data = report_data
        st.success("✅ Report generated successfully!")

        # Display automatic scan interpretation
        st.subheader("🔍 Automated Scan Interpretation")
        if tumour_type == "No Tumour":
            st.success("**SCAN RESULT: NORMAL** - No tumour detected in the MRI scan.")
        else:
            st.warning(f"**SCAN RESULT: ABNORMAL** - {tumour_type} detected with {confidence_score:.2f}% confidence.")

        # Display report preview
        st.subheader("📋 Report Preview")
        text_report = generate_text_report(report_data)
        st.text_area("Report Content", text_report, height=400, key="report_preview")

    # -----------------------------
    # Download Section
    # -----------------------------
    if st.session_state.report_data:
        st.subheader("📥 Download Report")

        col1, col2, col3 = st.columns(3)

        with col1:
            # Download as JSON
            json_report = json.dumps(st.session_state.report_data, indent=2)
            st.download_button(
                label="📄 Download JSON Report",
                data=json_report,
                file_name=f"brain_tumour_report_{st.session_state.report_data['patient_id']}.json",
                mime="application/json"
            )

        with col2:
            # Download as Text
            text_report = generate_text_report(st.session_state.report_data)
            st.download_button(
                label="📝 Download Text Report",
                data=text_report,
                file_name=f"brain_tumour_report_{st.session_state.report_data['patient_id']}.txt",
                mime="text/plain"
            )

        with col3:
            # Download analysis summary as CSV
            csv_data = pd.DataFrame([{
                'Patient_ID': st.session_state.report_data['patient_id'],
                'Scan_Result': st.session_state.report_data['scan_result'],
                'Tumour_Type': st.session_state.report_data['tumour_type'],
                'Confidence_Score': st.session_state.report_data['confidence_score'],
                'Tumour_Area_pixels': st.session_state.report_data['tumour_area'],
                'Tumour_Area_cm2': round(st.session_state.report_data['tumour_area'] * 0.0025, 2),
                'Analysis_Date': st.session_state.report_data['report_date']
            }])

            csv_string = csv_data.to_csv(index=False)
            st.download_button(
                label="📊 Download CSV Summary",
                data=csv_string,
                file_name=f"brain_tumour_summary_{st.session_state.report_data['patient_id']}.csv",
                mime="text/csv"
            )

# -----------------------------
# Optional: Confusion Matrix
# -----------------------------
st.subheader("📊 Evaluate Model (Optional)")
true_label = st.text_input("Enter True Label (for testing confusion matrix)", "")

if true_label and clf_model is not None and 'pred_class' in locals():
    labels = ['Glioma', 'Meningioma', 'No Tumour','Pituitary']
    if true_label not in labels:
        st.error(f"❌ Invalid label. Choose from: {', '.join(labels)}")
    else:
        y_true = [labels.index(true_label)]
        y_pred = [pred_class]
        cm = confusion_matrix(y_true, y_pred, labels=range(len(labels)))
        fig, ax = plt.subplots(figsize=(5, 5))
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
        disp.plot(ax=ax)
        st.pyplot(fig)

# -----------------------------
# Footer
# -----------------------------
st.markdown("---")
st.markdown("### 💡 Features Included:")
st.markdown("""
- ✅ **Automatic Tumour Detection** with clear scan interpretation
- ✅ **Tumour Classification** with confidence scores
- ✅ **Tumour Segmentation** with area calculation
- ✅ **Bounding Box** detection and visualization
- ✅ **Automated Clinical Notes** based on scan findings
- ✅ **Comprehensive Reporting** with multiple download options
- ✅ **Patient Information** management
- ✅ **Quantitative Analysis** of tumour characteristics
""")

st.markdown("### 🔍 Automated Scan Interpretation:")
st.markdown("""
- **NORMAL SCAN**: No tumour detected
- **ABNORMAL SCAN**: Tumour detected with type classification
- **Automatic clinical notes** generated based on findings
- **Tumour localization** and size estimation
- **Professional report format** suitable for medical use
""")

Writing app5.py


In [9]:
from pyngrok import ngrok

# paste your own token here
!ngrok authtoken "your_authtoken"


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [10]:

from pyngrok import ngrok


# Open tunnel
public_url = ngrok.connect(8501)
print("Streamlit URL:", public_url)

# Run streamlit
!streamlit run app5.py --server.address 0.0.0.0 --server.port 8501


Streamlit URL: NgrokTunnel: "https://ef570702aacd.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  URL: http://0.0.0.0:8501

2026-01-09 09:44:57.541432: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767951897.562681    3198 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767951897.569084    3198 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767951897.586245    3198 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767951897.586271    3198 computation_placer.cc:177] computation place